In [1]:
options(java.parameters = "-Xmx32000m")
library(loadeR)
library(transformeR)
library(convertR)

Loading required package: rJava
Loading required package: loadeR.java
Java version 1.8x amd64 by Azul Systems, Inc. detected
NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready
loadeR version 1.4.15 (2019-07-30) is loaded
Please use 'citation("loadeR")' to cite this package.
transformeR version 1.5.1 (2019-07-13) is loaded
Get the latest stable version (1.6.0) using <devtools::install_github('SantanderMetGroup/transformeR')>
Please see 'citation("transformeR")' to cite this package.
Loading required package: udunits2
udunits system database read from /opt/conda/lib/R/library/udunits2/share/udunits2.xml
convertR version 0.1.2 (2018-06-28) is loaded
  More information about the 'climate4R' ecosystem in: http://meteo.unican.es/climate4R

Attaching package: ‘convertR’

The following objects are masked from ‘package:loadeR’:

    hurs2huss, huss2hurs



In [2]:
# Output path where the data will be saved (change to your local path).
dir.data <- './data/' 
dir.Rdata <- './Rdata/'


#Defining the geographical domain
lonLim <- c(8, 13) 
latLim <- c(57, 62)

# Define the period and the season
years <- 1979:2019
season <- 1:12 #Full year

# Location of the lake
lake <- list(x= 10.7438771059,  y= 59.4392814052 )
lakename <- 'Vansjo'
    

# Define the dataset 
dataset <- "PIK_Obs-EWEMBI"

loadeR::loginUDG(username='WATExR',password="1234567890")

# Check available variables in the dataset (EWEMBI)  
di <- dataInventory(dataset)
names(di)

[2019-10-09 19:33:08] Setting credentials...
[2019-10-09 19:33:08] Success!
Go to <http://www.meteo.unican.es/udg-tap/home> for details on your authorized groups and datasets
[2019-10-09 19:33:09] Doing inventory ...
[2019-10-09 19:33:12] Retrieving info for 'orog' (14 vars remaining)
[2019-10-09 19:33:12] Retrieving info for 'hurs' (13 vars remaining)
[2019-10-09 19:33:13] Retrieving info for 'huss' (12 vars remaining)
[2019-10-09 19:33:13] Retrieving info for 'pr' (11 vars remaining)
[2019-10-09 19:33:13] Retrieving info for 'prsn' (10 vars remaining)
[2019-10-09 19:33:13] Retrieving info for 'ps' (9 vars remaining)
[2019-10-09 19:33:13] Retrieving info for 'psl' (8 vars remaining)
[2019-10-09 19:33:14] Retrieving info for 'rlds' (7 vars remaining)
[2019-10-09 19:33:14] Retrieving info for 'rsds' (6 vars remaining)
[2019-10-09 19:33:14] Retrieving info for 'wss' (5 vars remaining)
[2019-10-09 19:33:14] Retrieving info for 'tas' (4 vars remaining)
[2019-10-09 19:33:14] Retrieving info

[1] "orog"   "hurs"   "huss"   "pr"     "prsn"   "ps"     "psl"    "rlds"  
 [9] "rsds"   "wss"    "tas"    "tasmax" "tasmin" "uas"    "vas"

In [3]:
# Define the variables to be loaded. Remove those not needed. 
variables <- c("uas", "vas", "ps", "tas", "pr", "rsds", "rlds", "hurs")

In [4]:
C4R.vocabulary()

identifier,standard_name,units
<fct>,<fct>,<fct>
hurs,2-meter relative humidity,%
hursmax,maximum 2-meter relative humidity,%
hursmin,minimum 2-meter relative humidity,%
huss,2-meter specific humidity,kg.kg-1
hussmax,maximum 2-meter specific humidity,kg.kg-1
hussmin,minimum 2-meter specific humidity,kg.kg-1
lm,land binary mask,1
orog,surface altitude,m
ps,air pressure at surface level,Pa


In [5]:
#psl <-loadGridData(dataset, var = psl, years = 2010:2010, 
#                      lonLim = lonLim, latLim = latLim, 
#                      season = season)

In [6]:
# Load observations (EWEMBI) with function loadGridData from package loadeR.
# Data is loaded in a loop (function lapply) to load all variables in a single code line.
# A list of grids is obtained, each slot in the list corresponds to a variable
data.prelim <- lapply(variables, function(x) loadGridData(dataset, var = x, years = years, 
                                                   lonLim = lonLim, latLim = latLim, 
                                                   season = season))
names(data.prelim) <- variables

NOTE: Accessing harmonized data from a public UDG dataset
[2019-10-09 19:33:15] Defining harmonization parameters for variable "uas"
[2019-10-09 19:33:15] Opening dataset...
[2019-10-09 19:33:16] The dataset was successfuly opened
[2019-10-09 19:33:16] Defining geo-location parameters
[2019-10-09 19:33:17] Defining time selection parameters
Warning message in getTimeDomain(grid, dic, season, years, time, aggr.d, aggr.m, :
“Year selection out of boundaries. Only available years will be returned”[2019-10-09 19:33:17] Retrieving data subset ...
[2019-10-09 19:37:25] Done
NOTE: Accessing harmonized data from a public UDG dataset
[2019-10-09 19:37:25] Defining harmonization parameters for variable "vas"
[2019-10-09 19:37:25] Opening dataset...
[2019-10-09 19:37:26] The dataset was successfuly opened
[2019-10-09 19:37:26] Defining geo-location parameters
[2019-10-09 19:37:27] Defining time selection parameters
Warning message in getTimeDomain(grid, dic, season, years, time, aggr.d, aggr.m, :

In [7]:
# Bilinear interpolation of the data to the location of the lake. See ?interpGrid for other methods.
data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
                                                   method = "bilinear", 
                                                   bilin.method = "akima"))

#Convert pressure units to millibars with function udConvertGrid from package convertR.
data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

# Collect some common metadata (e.g. from variable uas)
dates <- data.interp[[1]]$Dates
xycoords <- getCoordinates(data.interp[[1]])

# Compute cloud cover with function rad2cc from package convertR
clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
clt$Variable$varName <- "cc"

# Put all variables together
data <- c(data.interp, "cc" = list(clt))

NOTE: New coordinates are irregular, bilin.method = 'fields'
[2019-10-09 20:06:20] Performing bilinear interpolation... may take a while
[2019-10-09 20:06:29] Done
NOTE: New coordinates are irregular, bilin.method = 'fields'
[2019-10-09 20:06:29] Performing bilinear interpolation... may take a while
[2019-10-09 20:06:38] Done
NOTE: New coordinates are irregular, bilin.method = 'fields'
[2019-10-09 20:06:38] Performing bilinear interpolation... may take a while
[2019-10-09 20:06:50] Done
NOTE: New coordinates are irregular, bilin.method = 'fields'
[2019-10-09 20:06:51] Performing bilinear interpolation... may take a while
[2019-10-09 20:07:02] Done
NOTE: New coordinates are irregular, bilin.method = 'fields'
[2019-10-09 20:07:03] Performing bilinear interpolation... may take a while
[2019-10-09 20:07:12] Done
NOTE: New coordinates are irregular, bilin.method = 'fields'
[2019-10-09 20:07:12] Performing bilinear interpolation... may take a while
[2019-10-09 20:07:26] Done
NOTE: New coordi

In [8]:
# save Rdata for posterior bias correction of seasonal forecasts
save(data, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), ".rda"))

In [10]:
install.packages('rlist')
library("rlist")
list.save(data, "ewembi.rds")

also installing the dependency ‘XML’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
